# Melee Stages VAE

In [3]:
from pathlib import Path
from os import listdir
from PIL import Image
import numpy as np
import cv2
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from google.colab import drive


drive.mount('/content/drive')

data_path = Path('/content/drive/My Drive/Melee/stage snippets')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
def resize(im, desired_size, maintain_aspect_ratio=True, padding_color=(0, 0, 0)):
    # adapted from https://jdhao.github.io/2017/11/06/resize-image-to-square-with-padding/#using-opencv
    old_size = im.shape[:2] # old_size is in (height, width) format

    ratio = float(desired_size)/max(old_size)
    new_size = tuple([int(x*ratio) for x in old_size])

    # new_size should be in (width, height) format

    im = cv2.resize(im, (new_size[1], new_size[0]))

    delta_w = desired_size - new_size[1]
    delta_h = desired_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)

    new_im = cv2.copyMakeBorder(im, top, bottom, left, right, cv2.BORDER_CONSTANT,
        value=padding_color)

    return new_im

def preprocess(im):
    return resize(im, 128)

In [ ]:
def get_stages_data(preprocess=lambda x: x, max_per_dir=10):
    imgs, labels = [], []
    for stage in listdir(data_path):
        folder_path = (data_path / stage).resolve()
        img_paths = [folder_path / img_filename for img_filename in listdir(data_path / folder_path)]
        n_imgs = len(img_paths)

        new_imgs = [np.asarray(Image.open(path)) for path in img_paths[:max_per_dir]]
        n_imgs = len(new_imgs)
        print(stage, n_imgs)
        preprocessed_imgs = list(map(preprocess, new_imgs))
        stage_imgs = np.array(list(map(np.asarray, preprocessed_imgs)))

        imgs.extend(stage_imgs)
        labels.extend([stage]*n_imgs)
    return np.array(imgs), np.array(labels)

def get_ordinal_encoder(labels):
    labels = sorted(set(labels))
    n_unique = len(set(labels))
    encoding = dict(list(zip(labels, list(range(n_unique)))))
    def encoder(label):
        return encoding[label]
    return encoder

In [9]:
X, y = get_stages_data(preprocess)

# encode stage names
ord_encoder = get_ordinal_encoder(y)
y_ord = np.array(list(map(ord_encoder, y)))
y_ohe = to_categorical(y_ord)

FoD_snippets 10
FD_snippets 10
PS_snippets 10
BF_snippets 10


In [10]:
print(X.shape)
print(y.shape)

(40, 128, 128, 3)
(40,)


In [ ]:

# The data, shuffled and split between train and test sets:
(X_train, _), (X_test, _) = train_test_split(X, y, train_size=0.8, shuffle=True, random_state=42)

X_train = X_train.astype('float32') / 255
X_test = X_test.astype('float32') / 255

print(X_train.shape)
print(X_test.shape)

In [ ]:
input_shape = X_train[0].shape
latent_dim = 1024

encoder = get_encoder(input_shape, latent_dim)
#print(encoder.summary())

decoder = get_decoder(latent_dim)
#print(decoder.summary())

vae = VAE(encoder, decoder)
vae.compile(optimizer=Adam())

vae.fit(X_train, epochs=30, batch_size=32)

In [ ]:
view_images(X_test[:10], decoder, encoder)